In [1]:
import requests
import pandas as pd 
import numpy as np
import json
import time
import sys

api_key = "6df661d9964de19e4ca7721d9cce69a6"
url_api = "https://api.themoviedb.org/3"

In [3]:
# Construction de la database

columns = ['date', 'titre', 'budget', 'recette', 'duree', 'genres', 'synopsis', 'tagline', 'realisateur', 'casting', 'compagnies_production', 'suite', 'production_countries', 'langues', 'imdb_id', 'vote_average', 'vote_count']

def build_base(start, end):
    array = []
    
    for id in range(start, end):
        response = get_movie_id(id)
        
        if response.status_code == requests.codes.OK :
            dict = json.loads(response.text)
            features = []
            if dict["budget"] != 0 and dict["revenue"] != 0:
                #On ajoute les features
                features.append(dict["release_date"])
                features.append(dict["original_title"])
                features.append(dict["budget"])
                features.append(dict["revenue"])
                

                features.append(dict["runtime"])

                genres = []
                for ele in dict["genres"]:
                    genres.append(ele['name'])

                features.append(genres)
                features.append(dict['overview'])
                features.append(dict["tagline"])


                #Casting, réalisateur
                response = get_casting_id(id)
                cast = []
                realisateur = []
                if response.status_code == requests.codes.OK :
                    dict_casting = json.loads(response.text)
                    for ele in dict_casting["cast"]:
                        if ele["known_for_department"] == "Acting":
                            cast.append(ele["name"])
                    
                    for ele in dict_casting["crew"]:
                        if ele["known_for_department"] == "Directing" or ele["department"] == 'Directing':
                            if "job" in ele.keys():
                                if ele["job"] == "Director":
                                    realisateur.append(ele["name"])
                features.append(realisateur)
                features.append(cast)


                #Compagnies_production
                compagnies_production = []
                for ele in dict["production_companies"]:
                    compagnies_production.append(ele["name"])
                features.append(compagnies_production)


                #Suite
                if dict["belongs_to_collection"] != None:
                    response = get_collections(dict["belongs_to_collection"]["id"])
                    if response.status_code == requests.codes.OK :
                        dict_collection = json.loads(response.text)
                        k = 1
                        for ele in dict_collection["parts"]:
                            if ele["original_title"] == dict["original_title"]:
                                features.append(k)
                                break
                            k += 1
                else : 
                    features.append(1)
                #Production_countries
                production_countries = []
                for ele in dict['production_countries']:
                    production_countries.append(ele["iso_3166_1"])
                
                features.append(production_countries)


                #Langues
                langues = []
                for ele in dict["spoken_languages"]:
                    langues.append(ele["iso_639_1"])
                
                features.append(langues)

                #imdb_id

                features.append(dict["imdb_id"])

                #vote_average
                features.append(dict["vote_average"])

                #vote_count
                features.append(dict["vote_count"])





                #Ajout ligne
                array.append(features)



    df = pd.DataFrame(np.array(array),columns=columns)
    df.to_csv(f"csv/{start}_{end}.csv")

    return 


In [4]:
#Construction

def build_all_base(debut):
    for k in range(debut, 1, -1000):
        try :
            if k==1:
                build_base(1, 1000)
            else: 
                build_base(k-1000, k)
        except :
            print("Not data")
            print(sys.exc_info())

In [ ]:
build_all_base(534000)

In [5]:
#Fusion des csv
# importing libraries
import pandas as pd
import glob
import os
  
# merging the files
joined_files = os.path.join("csv", "*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df = df.drop(columns="Unnamed: 0")
df.to_csv("csv/total.csv")



In [ ]:
df = pd.read_csv('csv/total.csv', index_col=0)
df

In [5]:
import json
import pandas as pd


name = []
popularity = []
with open('data/person_ids_11_09_2022.json', encoding='utf-8') as f:
    for line in f:
        json_line = json.loads(line)
        if json_line["popularity"] > 1 :
            name.append(json_line["name"])
            popularity.append(json_line["popularity"])
        


df = pd.DataFrame((zip(name, popularity)), columns = ['Name', 'Popularity'])
df.head()
df.to_csv("people.csv")

In [2]:
import pandas as pd

df = pd.read_csv("data/people.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df.describe()

,Popularity
count,521375.000000
mean,2.140541
std,2.380080
min,1.001000
25%,1.380000
50%,1.487000
75%,1.960000
max,157.976000
